## **Variational Autoencoder (VAE)**

Variational Autoencoders (VAE) are a probabilistic extension of autoencoders. They learn a distribution over the latent space instead of a deterministic point. This allows VAEs to generate new data by sampling from the latent space, making them useful for generative tasks.

**Imports**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np


**Data Loading**

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Minimal Preprocessing**

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

**Variational Autoencoder Model**

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, hidden_dim)
        self.fc22 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)  # Mean and log variance

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        return torch.sigmoid(self.fc3(z))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 20))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Instantiate the model
model = VAE(input_dim=X_train.shape[1], hidden_dim=8)


**Train the Model**

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss(reduction='sum')

def loss_function(recon_x, x, mu, logvar):
    BCE = loss_fn(recon_x, x.view(-1, 20))
    # KL divergence
    MSE = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + MSE

for epoch in range(50):
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(torch.tensor(X_train, dtype=torch.float32))
    loss = loss_function(recon_batch, torch.tensor(X_train, dtype=torch.float32), mu, logvar)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}")